In [26]:
import numpy as np
import scipy as sp
from numba import jit
import matplotlib.pyplot as plt

In [20]:
@jit
def dprod(A,B):
    nobsa,na= A.shape
    nobsb,nb= B.shape
    Res= np.empty((nobsa,nb*na))
    if (nobsa!=nobsb):
        return 'A and B must have same number of rows'
    for ia in range(na):
        for ib in range(nb):
            for t in range(nobsa):
                Res[t,nb*(ia-1)+ib]=A[t,ia] * B[t, ib]
    return Res

In [3]:
@jit
def Chebnodes(p, nodetype=0):
    n, a, b = p[0], p[1], p[2]
    s = (b-a) / 2 
    m = (b+a) / 2  
    if (nodetype < 2):  # usual nodes
        k = np.pi*np.linspace(0.5,n-0.5,n)  
        x = m - np.cos(k[0:n]/n) * s  
        if (nodetype == 1):  # Extend nodes to endpoints
            aa = x[0]  
            bb = x[-1]  
            x = (bb*a - aa*b)/(bb-aa) + (b-a)/(bb-aa)*x
    else: # Lobatto nodes
        k = np.pi*np.linspace(0,n-1,n)
        x = m - np.cos(k[0:n]/(n-1)) * s
    return x

In [22]:
@jit
def Chebbasex(p, x):
    n, a, b = p[0], p[1], p[2]
    z = (2/(b-a)) * (x-(a+b)/2)
    m = z.shape[0]
    bas = np.empty((m, n));
    bas[:, 0] = 1.0
    bas[:, 1] = z[:]
    z = 2 * z
    for j in range(2,n):
        for i in range(m):
            bas[i, j] = z[i] * bas[i, j-1] - bas[i, j-2]
    return bas

In [24]:
@jit
def Monobasex(p, x):  # Monomials basis matrix
    n, a, b = p[0], p[1], p[2]
    z = (2/(b-a)) * (x-(a+b)/2)
    m = z.shape[0]
    bas = np.empty((m, n));
    bas[:, 0] = 1.0
    bas[:, 1] = z[:]
    for j in range(2,n):
        for i in range(m):
            bas[i, j] = z[i] * bas[i, j-1]
    return bas

In [58]:
@jit
def ChebDiffMatrix(p):  # Neat code from gregvw/chebyshev-methods/chebdiff.py on github
    n, a, b = p[0], p[1], p[2]  #Differentiating matrix for Chebyshev polynomials
    k= np.arange(n)
    a= (-1)**k
    A= sp.triu(1-np.outer(a,a))
    D= np.dot(A,np.diag(k))
    D[0,:] = D[0,:]/2
    D= D/(b-a)*2
    return D

In [ ]:
@jit
def MonoDiffMatrix(p):  
    n, a, b = p[0], p[1], p[2]  #Differentiating matrix for mononomials - not finished yet
    k= np.arange(n)
    a= (-1)**k
    A= sp.triu(1-np.outer(a,a))
    D= np.dot(A,np.diag(k))
    D[0,:] = D[0,:]/2
    D= D/(b-a)*2
    return D

In [21]:
%%timeit
A1=np.random.randn(10,10)
B1=np.random.randn(10,100000)
C1=dprod(A1,B1)

The slowest run took 7.82 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 165 ms per loop


In [ ]:
A= np.ones((2,3))
B= 2*np.ones((2,2))
C= np.empty((2,))
na,na1=A.shape
na

In [15]:
C1.shape

(10, 1000000)

In [61]:
#Approximate cos(x) with Chebychev
n=12
x=Chebnodes((n,0,20),2)
y= np.cos(x)
Phi=Chebbasex((n,0,20),x)
#Invert to get polynomial coefficients
coeff= np.linalg.inv(Phi) @ y
D=ChebDiffMatrix((n,0,20))
x1=Chebnodes((100,0,20),0)
y1=np.cos(x1)
z1=-np.cos(x1)
Phi1=Chebbasex((n,0,20),x1)
y_approx= Phi1 @ coeff
z_approx= Phi1 @ D @ D @ coeff
plt.plot(x1,z1)
plt.plot(x1,z_approx)
plt.show()

In [25]:
#Approximate cos(x) with monomial
x=Chebnodes((10,0,20),2)
y= np.cos(x)
Phi=Monobasex((10,0,20),x)
#Invert to get polynomial coefficients
coeff= np.linalg.inv(Phi) @ y

x1=Chebnodes((100,0,20),0)
y1=np.cos(x1)
Phi1=Monobasex((10,0,20),x1)
y_approx= Phi1 @ coeff
plt.plot(x1,y1)
plt.plot(x1,y_approx)
plt.show()

In [34]:
D1=modalDiffMatrix(10)

In [35]:
D1

array([[ 0,  1,  0,  3,  0,  5,  0,  7,  0,  9],
       [ 0,  0,  4,  0,  8,  0, 12,  0, 16,  0],
       [ 0,  0,  0,  6,  0, 10,  0, 14,  0, 18],
       [ 0,  0,  0,  0,  8,  0, 12,  0, 16,  0],
       [ 0,  0,  0,  0,  0, 10,  0, 14,  0, 18],
       [ 0,  0,  0,  0,  0,  0, 12,  0, 16,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 14,  0, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 16,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])